In [3]:
import numpy as np
import pandas as pd
import numpy as np
from finta import TA
import yfinance as yf
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('labels_2017_2019.csv')
data

,Unnamed: 0,symbol,message,datetime,user,message_id,Date,Time,label,Bearish,Nay,Neutral,Bullish,To the Moon!!
0,159223,AAPL,aapl would short pull back I sure hits 350,2019-12-31 23:57:46,941820,188613177,2019-12-31,23:57:46,1,-2.630906,-2.177632,-1.390408,7.412164,-0.539381
1,159224,AAPL,aapl 300 going happen could selling markets ge...,2019-12-31 23:50:02,1129754,188612780,2019-12-31,23:50:02,1,-1.727318,-2.461119,-2.118949,-0.295749,8.159695
2,159225,AAPL,aapl new article picking spots income alternat...,2019-12-31 23:48:05,1555408,188612676,2019-12-31,23:48:05,1,-1.903158,-0.768996,7.448404,-0.619132,-2.674681
3,159226,AAPL,spy china deal going bs trump pushed amp pumpe...,2019-12-31 23:43:24,543250,188612375,2019-12-31,23:43:24,1,7.364426,-0.456597,-2.297388,-2.874764,-2.173209
4,159227,AAPL,spy 50 stock market held top 1 got richer funn...,2019-12-31 23:39:33,543250,188612175,2019-12-31,23:39:33,1,-1.269605,-2.315129,-1.728028,-1.411539,7.940353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132196,2263445,NFLX,netflix became drug 2016 nflx,2017-01-01 19:11:41,13885,70638109,2017-01-01,19:11:41,1,-1.674075,-1.270964,7.254238,-0.330992,-2.220495
1132197,2263446,NFLX,nflx would short short term pt 100,2017-01-01 05:27:05,817817,70632488,2017-01-01,05:27:05,1,-1.633899,-1.251419,7.555491,-0.967629,-2.600437
1132198,2263447,NFLX,nflx chill back show,2017-01-01 05:09:33,886324,70632281,2017-01-01,05:09:33,1,-2.480931,1.014106,6.573313,-0.177456,-4.106884
1132199,2263448,NFLX,netflix became drug 2016 nflx,2017-01-01 02:34:55,13885,70631332,2017-01-01,02:34:55,1,-1.674075,-1.270964,7.254238,-0.330992,-2.220495


In [81]:
data = pd.read_csv('labels_2017_2019.csv')

data['symbol'] = data['symbol'].replace({'FB':'META'})
codes = data['symbol'].unique()
start_date = data['Date'].min()
end_date = data['Date'].max()
res = []


for ticker_symbol in tqdm(codes):
    ticker = yf.Ticker(ticker_symbol)
    df = ticker.history(start=start_date, end=end_date)
    df['symbol'] = ticker_symbol
    res.append(df)

    
price = pd.concat(res).reset_index()
price['Time'] = price['Date'].dt.strftime('%H:%M:%S')
price['Date'] = price['Date'].dt.strftime('%Y-%m-%d')
price['Pctchange'] = (price['Close'] - price['Open'])/price['Open']
price_new = pd.DataFrame(price.values[:,1:], index = price['Date'].values[:], columns = price.columns[1:]).reset_index()
price_new.columns = price.columns
sentiment = pd.merge(data, price_new, left_on=['Date','symbol'], right_on=['Date','symbol'], how='outer')
sentiment = sentiment.sort_values('Date')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.22it/s]


In [82]:
codes = sentiment.symbol.unique()
res = []
for code in codes:
    df_temp = sentiment[sentiment['symbol']==code]
    df_temp['Pctchange'] = df_temp['Pctchange'].fillna(method='bfill')
    df_temp = df_temp[['message','symbol','Date','Pctchange','Open','High','Low','Close','Volume','Dividends','Stock Splits', 'Bearish', 'Nay',
       'Neutral', 'Bullish', 'To the Moon!!']].dropna()
    res.append(df_temp)

In [83]:
data_final = pd.concat(res)

In [84]:
data_final = data_final[(data_final['Date']>='2017-01-01')&(data_final['Date']<='2019-12-31')]

In [85]:
data_final.to_csv('data_2017_2019_with_price.csv')